In [54]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# remove rows without invoice number and convert to string
df = pd.read_excel('Online Retail.xlsx')
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

# removing credit invoices / returns
df = df[~df['InvoiceNo'].str.contains('C')]

# filtering to United kingdom and grouping by invoice number and description, to show quantity of each item per order
market_basket = df[df['Country'] =="United Kingdom"].groupby(
                ['InvoiceNo', 'Description'])['Quantity']

# summing quantities of each item, item descriptions as columns and invoice number as row index
market_basket = market_basket.sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
market_basket.head()



Description,20713,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
def encode_data(datapoint):
    if datapoint <= 0:
        return 0
    if datapoint >= 1:
        return 1
    
market_basket = market_basket.applymap(encode_data)

In [25]:
itemsets = apriori(market_basket, min_support=0.02, use_colnames=True)


In [52]:
rules = association_rules(itemsets, metric="lift", min_threshold=1)
rules.count()

antecedents           88
consequents           88
antecedent support    88
consequent support    88
support               88
confidence            88
lift                  88
leverage              88
conviction            88
dtype: int64

In [53]:
def get_consequents(input_item):
    consequents = []
    for i in range(len(rules)):
        for item in rules['antecedents'].iloc[i]:
            if item == input_item:
                for product in rules['consequents'].iloc[i]:
                    consequents.append(product)
    consequents = list(dict.fromkeys(consequents))
    print(consequents)



get_consequents('PINK REGENCY TEACUP AND SAUCER')

['GREEN REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER ']
